# Module 4: Supervised Learning — Regression

---

Regression is the branch of supervised learning where we predict a **continuous numerical value**. This module covers the most important regression algorithms, from simple linear regression to regularized methods, along with evaluation metrics and practical code.

**What you will learn:**
- Simple and Multiple Linear Regression
- Polynomial Regression
- Regularization: Ridge, Lasso, and ElasticNet
- Gradient Descent from scratch
- Regression evaluation metrics

---

## Table of Contents

1. [Simple Linear Regression](#1.-Simple-Linear-Regression)
2. [Multiple Linear Regression](#2.-Multiple-Linear-Regression)
3. [Polynomial Regression](#3.-Polynomial-Regression)
4. [Regularization: Ridge, Lasso, ElasticNet](#4.-Regularization)
5. [Gradient Descent from Scratch](#5.-Gradient-Descent-from-Scratch)
6. [Regression Metrics](#6.-Regression-Metrics)
7. [Exercises](#7.-Exercises)
8. [Summary and Further Reading](#8.-Summary-and-Further-Reading)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

---

## 1. Simple Linear Regression

Simple linear regression models the relationship between a single feature (x) and a target (y) as a straight line:

$$y = w_0 + w_1 \cdot x$$

Where:
- $w_0$ is the **intercept** (bias)
- $w_1$ is the **slope** (weight/coefficient)

The model finds the line that minimizes the sum of squared differences between predicted and actual values.

In [ ]:
# Generate synthetic data: study hours vs exam score
np.random.seed(42)
hours = np.random.uniform(1, 10, 50)
scores = 10 + 8 * hours + np.random.normal(0, 5, 50)

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(hours, scores, color='#2196F3', s=60, edgecolors='white', linewidth=0.5)
ax.set_xlabel('Study Hours', fontsize=13)
ax.set_ylabel('Exam Score', fontsize=13)
ax.set_title('Study Hours vs Exam Score', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("There appears to be a positive linear relationship between study hours and exam scores.")

In [ ]:
from sklearn.linear_model import LinearRegression

# Reshape X for sklearn (needs 2D array)
X = hours.reshape(-1, 1)
y = scores

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
lr = LinearRegression()
lr.fit(X_train, y_train)

print("Simple Linear Regression Results:")
print(f"  Intercept (w0):  {lr.intercept_:.4f}")
print(f"  Slope (w1):      {lr.coef_[0]:.4f}")
print(f"  Equation:        y = {lr.intercept_:.2f} + {lr.coef_[0]:.2f} * x")
print(f"  Training R2:     {lr.score(X_train, y_train):.4f}")
print(f"  Test R2:         {lr.score(X_test, y_test):.4f}")

In [ ]:
# Plot the regression line with predictions
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left: Regression line
ax = axes[0]
ax.scatter(X_train, y_train, color='#2196F3', s=60, edgecolors='white',
           linewidth=0.5, label='Training data')
ax.scatter(X_test, y_test, color='#FF5722', s=60, edgecolors='white',
           linewidth=0.5, marker='s', label='Test data')
x_line = np.linspace(0, 11, 100).reshape(-1, 1)
ax.plot(x_line, lr.predict(x_line), 'k-', linewidth=2, label='Regression line')
ax.set_xlabel('Study Hours', fontsize=13)
ax.set_ylabel('Exam Score', fontsize=13)
ax.set_title('Linear Regression Fit', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)

# Right: Residual plot
ax = axes[1]
y_pred_train = lr.predict(X_train)
residuals = y_train - y_pred_train
ax.scatter(y_pred_train, residuals, color='#2196F3', s=60, edgecolors='white', linewidth=0.5)
ax.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax.set_xlabel('Predicted Values', fontsize=13)
ax.set_ylabel('Residuals', fontsize=13)
ax.set_title('Residual Plot', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("A good residual plot shows randomly scattered points around zero,")
print("with no obvious pattern. Patterns would suggest a non-linear relationship.")

---

## 2. Multiple Linear Regression

When we have multiple features, the model becomes:

$$y = w_0 + w_1 x_1 + w_2 x_2 + \ldots + w_p x_p$$

This is the same concept extended to higher dimensions. We will use a real-world dataset for this section.

In [ ]:
from sklearn.datasets import fetch_california_housing

# Load the California Housing dataset
housing = fetch_california_housing()
X_housing = pd.DataFrame(housing.data, columns=housing.feature_names)
y_housing = housing.target  # Median house value in $100K

print(f"Dataset shape: {X_housing.shape}")
print(f"Target: Median house value (in $100,000)")
print(f"\nFeature descriptions:")
for name in housing.feature_names:
    print(f"  - {name}")
print(f"\n{X_housing.describe().round(2)}")

In [ ]:
# Split and scale
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(
    X_housing, y_housing, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_h)
X_test_scaled = scaler.transform(X_test_h)

# Train multiple linear regression
mlr = LinearRegression()
mlr.fit(X_train_scaled, y_train_h)

print("Multiple Linear Regression on California Housing:")
print(f"  Training R2: {mlr.score(X_train_scaled, y_train_h):.4f}")
print(f"  Test R2:     {mlr.score(X_test_scaled, y_test_h):.4f}")
print(f"\nFeature coefficients:")
coef_df = pd.DataFrame({
    'Feature': housing.feature_names,
    'Coefficient': mlr.coef_
}).sort_values('Coefficient', key=abs, ascending=False)
print(coef_df.to_string(index=False))

In [ ]:
# Visualize feature importance (coefficients)
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['#2196F3' if c >= 0 else '#FF5722' for c in coef_df['Coefficient']]
ax.barh(coef_df['Feature'], coef_df['Coefficient'], color=colors, edgecolor='white')
ax.set_xlabel('Coefficient Value', fontsize=13)
ax.set_title('Feature Coefficients (Multiple Linear Regression)', fontsize=14, fontweight='bold')
ax.axvline(x=0, color='black', linewidth=0.5)
plt.tight_layout()
plt.show()

print("Positive coefficients increase the predicted price; negative ones decrease it.")
print("Note: coefficients are comparable because we scaled the features first.")

---

## 3. Polynomial Regression

When the relationship between features and target is non-linear, we can add polynomial terms (x^2, x^3, etc.) to capture curved patterns.

$$y = w_0 + w_1 x + w_2 x^2 + w_3 x^3 + \ldots$$

Despite the non-linear features, this is still "linear" regression — it is linear in the coefficients.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

# Generate non-linear data
np.random.seed(42)
X_nl = np.sort(np.random.uniform(-3, 3, 60)).reshape(-1, 1)
y_nl = 0.5 * X_nl[:, 0]**3 - 2 * X_nl[:, 0]**2 + X_nl[:, 0] + 3 + np.random.normal(0, 3, 60)

X_train_nl, X_test_nl, y_train_nl, y_test_nl = train_test_split(
    X_nl, y_nl, test_size=0.2, random_state=42
)

# Compare different polynomial degrees
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
degrees = [1, 2, 3, 10]
X_plot = np.linspace(-3.5, 3.5, 200).reshape(-1, 1)

for idx, degree in enumerate(degrees):
    ax = axes[idx]
    
    # Create polynomial features and fit
    poly = PolynomialFeatures(degree=degree)
    X_train_poly = poly.fit_transform(X_train_nl)
    X_test_poly = poly.transform(X_test_nl)
    X_plot_poly = poly.transform(X_plot)
    
    model = LinearRegression()
    model.fit(X_train_poly, y_train_nl)
    
    train_r2 = model.score(X_train_poly, y_train_nl)
    test_r2 = model.score(X_test_poly, y_test_nl)
    
    ax.scatter(X_train_nl, y_train_nl, color='#2196F3', s=40, edgecolors='white',
               linewidth=0.3, label='Train')
    ax.scatter(X_test_nl, y_test_nl, color='#FF5722', s=40, edgecolors='white',
               linewidth=0.3, marker='s', label='Test')
    y_plot = model.predict(X_plot_poly)
    y_plot = np.clip(y_plot, -50, 50)  # clip for visibility
    ax.plot(X_plot, y_plot, 'k-', linewidth=2)
    ax.set_title(f'Degree {degree}\nTrain R2={train_r2:.3f}, Test R2={test_r2:.3f}',
                 fontsize=11, fontweight='bold')
    ax.set_ylim(-30, 30)
    ax.legend(fontsize=8)

plt.suptitle('Polynomial Regression — Effect of Degree', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

print("Observations:")
print("  - Degree 1 (linear): underfits — cannot capture the curve.")
print("  - Degree 3: good fit — matches the true underlying relationship.")
print("  - Degree 10: overfits — fits noise in training data, performs poorly on test data.")

---

## 4. Regularization: Ridge, Lasso, and ElasticNet

Regularization adds a penalty term to the loss function to prevent overfitting by discouraging large coefficients.

| Method | Penalty | Effect |
|--------|---------|--------|
| **Ridge (L2)** | Sum of squared coefficients | Shrinks coefficients toward zero |
| **Lasso (L1)** | Sum of absolute coefficients | Can set coefficients exactly to zero (feature selection) |
| **ElasticNet** | Combination of L1 and L2 | Balances both approaches |

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet

# Use the California Housing dataset
models = {
    'Linear Regression': LinearRegression(),
    'Ridge (alpha=1.0)': Ridge(alpha=1.0),
    'Lasso (alpha=0.1)': Lasso(alpha=0.1),
    'ElasticNet (alpha=0.1)': ElasticNet(alpha=0.1, l1_ratio=0.5),
}

results = []
coefficients = {}

for name, model in models.items():
    model.fit(X_train_scaled, y_train_h)
    train_r2 = model.score(X_train_scaled, y_train_h)
    test_r2 = model.score(X_test_scaled, y_test_h)
    y_pred = model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test_h, y_pred))
    results.append({'Model': name, 'Train R2': train_r2, 'Test R2': test_r2, 'RMSE': rmse})
    coefficients[name] = model.coef_

results_df = pd.DataFrame(results)
print("Model Comparison:")
print(results_df.to_string(index=False))

In [ ]:
# Visualize how regularization affects coefficients
fig, ax = plt.subplots(figsize=(14, 6))
x_pos = np.arange(len(housing.feature_names))
width = 0.2
colors = ['#2196F3', '#4CAF50', '#FF9800', '#9C27B0']

for i, (name, coefs) in enumerate(coefficients.items()):
    ax.bar(x_pos + i * width, coefs, width, label=name, color=colors[i], edgecolor='white')

ax.set_xticks(x_pos + 1.5 * width)
ax.set_xticklabels(housing.feature_names, rotation=45, ha='right')
ax.set_ylabel('Coefficient Value', fontsize=13)
ax.set_title('Coefficient Comparison Across Regularization Methods', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.axhline(y=0, color='black', linewidth=0.5)
plt.tight_layout()
plt.show()

print("Observations:")
print("  - Lasso drives some coefficients to exactly zero (feature selection).")
print("  - Ridge shrinks all coefficients but keeps them all non-zero.")
print("  - ElasticNet is a mixture of both behaviors.")

In [ ]:
# Effect of alpha (regularization strength) on Ridge coefficients
alphas = np.logspace(-2, 4, 50)
ridge_coefs = []

for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_scaled, y_train_h)
    ridge_coefs.append(ridge.coef_)

ridge_coefs = np.array(ridge_coefs)

fig, ax = plt.subplots(figsize=(12, 6))
for i, name in enumerate(housing.feature_names):
    ax.plot(alphas, ridge_coefs[:, i], linewidth=2, label=name)

ax.set_xscale('log')
ax.set_xlabel('Alpha (regularization strength)', fontsize=13)
ax.set_ylabel('Coefficient Value', fontsize=13)
ax.set_title('Ridge Coefficients as a Function of Alpha', fontsize=14, fontweight='bold')
ax.legend(fontsize=9, bbox_to_anchor=(1.05, 1), loc='upper left')
ax.axhline(y=0, color='black', linewidth=0.5, linestyle='--')
plt.tight_layout()
plt.show()

print("As alpha increases, all coefficients shrink toward zero.")
print("Small alpha: behaves like ordinary linear regression.")
print("Large alpha: heavily penalizes large coefficients, leading to underfitting.")

---

## 5. Gradient Descent from Scratch

Let us implement linear regression using gradient descent to understand how the optimization actually works.

In [ ]:
# Simple gradient descent for linear regression: y = w*x + b

# Use the study hours data from Section 1
X_gd = hours.reshape(-1, 1)
y_gd = scores

# Normalize X
X_mean, X_std = X_gd.mean(), X_gd.std()
X_norm = (X_gd - X_mean) / X_std

# Initialize parameters
w = 0.0  # weight
b = 0.0  # bias
learning_rate = 0.1
n_iterations = 100
n_samples = len(X_norm)

# Track loss history
loss_history = []
w_history = []
b_history = []

for i in range(n_iterations):
    # Forward pass: predictions
    y_pred = w * X_norm[:, 0] + b
    
    # Compute loss (Mean Squared Error)
    loss = np.mean((y_gd - y_pred) ** 2)
    loss_history.append(loss)
    w_history.append(w)
    b_history.append(b)
    
    # Compute gradients
    dw = -2/n_samples * np.sum(X_norm[:, 0] * (y_gd - y_pred))
    db = -2/n_samples * np.sum(y_gd - y_pred)
    
    # Update parameters
    w = w - learning_rate * dw
    b = b - learning_rate * db

print("Gradient Descent Results:")
print(f"  Final weight (w): {w:.4f}")
print(f"  Final bias (b):   {b:.4f}")
print(f"  Final MSE loss:   {loss_history[-1]:.4f}")
print(f"\nCompare with sklearn LinearRegression:")
lr_check = LinearRegression()
lr_check.fit(X_norm, y_gd)
print(f"  sklearn weight:   {lr_check.coef_[0]:.4f}")
print(f"  sklearn bias:     {lr_check.intercept_:.4f}")

In [ ]:
# Visualize gradient descent convergence
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loss over iterations
axes[0].plot(loss_history, linewidth=2, color='#FF5722')
axes[0].set_xlabel('Iteration', fontsize=13)
axes[0].set_ylabel('MSE Loss', fontsize=13)
axes[0].set_title('Loss Convergence', fontsize=14, fontweight='bold')

# Weight trajectory
axes[1].plot(w_history, linewidth=2, color='#2196F3')
axes[1].axhline(y=lr_check.coef_[0], color='red', linestyle='--', label='Optimal w')
axes[1].set_xlabel('Iteration', fontsize=13)
axes[1].set_ylabel('Weight (w)', fontsize=13)
axes[1].set_title('Weight Convergence', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)

# Final fit
axes[2].scatter(X_norm, y_gd, color='#2196F3', s=40, edgecolors='white', linewidth=0.3)
x_plot = np.linspace(X_norm.min(), X_norm.max(), 100)
axes[2].plot(x_plot, w * x_plot + b, 'r-', linewidth=2, label='Gradient descent fit')
axes[2].set_xlabel('Normalized Study Hours', fontsize=13)
axes[2].set_ylabel('Exam Score', fontsize=13)
axes[2].set_title('Final Gradient Descent Fit', fontsize=14, fontweight='bold')
axes[2].legend(fontsize=10)

plt.tight_layout()
plt.show()

---

## 6. Regression Metrics

Understanding how to measure model performance is just as important as building the model.

| Metric | Formula | Interpretation |
|--------|---------|---------------|
| **MAE** | Mean of absolute errors | Average magnitude of errors (in original units) |
| **MSE** | Mean of squared errors | Penalizes large errors more heavily |
| **RMSE** | Square root of MSE | Same units as target variable |
| **R2 Score** | 1 - (SS_res / SS_tot) | Proportion of variance explained (1.0 = perfect) |

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Use the California Housing multiple regression model
y_pred_housing = mlr.predict(X_test_scaled)

mae = mean_absolute_error(y_test_h, y_pred_housing)
mse = mean_squared_error(y_test_h, y_pred_housing)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_h, y_pred_housing)

print("Regression Metrics (California Housing — Linear Regression):")
print(f"  MAE:  {mae:.4f}  (average error: ${mae * 100000:,.0f})")
print(f"  MSE:  {mse:.4f}")
print(f"  RMSE: {rmse:.4f}  (typical error: ${rmse * 100000:,.0f})")
print(f"  R2:   {r2:.4f}  (explains {r2:.1%} of variance)")

In [ ]:
# Actual vs Predicted plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Actual vs Predicted
ax = axes[0]
ax.scatter(y_test_h, y_pred_housing, alpha=0.4, color='#2196F3', s=20, edgecolors='white', linewidth=0.2)
ax.plot([0, 5.5], [0, 5.5], 'r--', linewidth=2, label='Perfect prediction')
ax.set_xlabel('Actual Values', fontsize=13)
ax.set_ylabel('Predicted Values', fontsize=13)
ax.set_title('Actual vs Predicted (California Housing)', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)

# Residual distribution
ax = axes[1]
residuals_h = y_test_h - y_pred_housing
ax.hist(residuals_h, bins=40, color='#2196F3', edgecolor='white', alpha=0.7)
ax.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax.set_xlabel('Residual (Actual - Predicted)', fontsize=13)
ax.set_ylabel('Frequency', fontsize=13)
ax.set_title('Residual Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("A well-performing model shows points clustered tightly around the diagonal line,")
print("and residuals centered around zero with a roughly normal distribution.")

---

## 7. Exercises

### Exercise 1: Boston-Style Housing Regression

In [ ]:
# Exercise 1: Using the California Housing dataset:
# 1. Split the data (80/20) and scale the features
# 2. Train Linear Regression, Ridge (alpha=10), and Lasso (alpha=0.01)
# 3. Compute MAE, RMSE, and R2 for each model on the test set
# 4. Create a bar chart comparing RMSE across models
# 5. Which model performs best? Why?

# Your code here:


### Exercise 2: Polynomial Degree Selection

In [ ]:
# Exercise 2: For the non-linear data below:
# 1. Try polynomial degrees from 1 to 15
# 2. Record training and test R2 for each degree
# 3. Plot training R2 and test R2 vs degree on the same chart
# 4. Identify the optimal degree (highest test R2)
# 5. Comment on the overfitting behavior at high degrees

np.random.seed(42)
X_ex = np.sort(np.random.uniform(-2, 2, 80)).reshape(-1, 1)
y_ex = np.sin(2 * X_ex[:, 0]) + 0.3 * np.random.randn(80)

# Your code here:


### Exercise 3: Gradient Descent Variations

In [ ]:
# Exercise 3: Modify the gradient descent implementation from Section 5:
# 1. Try learning rates: 0.001, 0.01, 0.1, 0.5
# 2. Run each for 200 iterations
# 3. Plot the loss curves for all four learning rates on the same chart
# 4. Which learning rate converges the fastest?
# 5. Does any learning rate diverge?

# Your code here:


---

## 8. Summary and Further Reading

### What We Covered

- **Simple Linear Regression**: Fitting a line to data with one feature.
- **Multiple Linear Regression**: Extending to multiple features using the California Housing dataset.
- **Polynomial Regression**: Capturing non-linear relationships by adding polynomial terms.
- **Regularization**: Ridge (L2), Lasso (L1), and ElasticNet to prevent overfitting and perform feature selection.
- **Gradient Descent**: How optimization works under the hood.
- **Metrics**: MAE, MSE, RMSE, and R2 for evaluating regression models.

### Recommended Reading

- [Scikit-learn Linear Models Documentation](https://scikit-learn.org/stable/modules/linear_model.html)
- Chapter 4 of Aurélien Géron, *Hands-On Machine Learning* (Training Models)
- Chapter 3 and 6 of *Introduction to Statistical Learning (ISLR)* — Linear Regression and Regularization

### Next Module

In **Module 5: Supervised Learning — Classification**, we will cover the algorithms used when the target is a category rather than a number: Logistic Regression, KNN, SVM, Decision Trees, and Naive Bayes.

---